In [1]:
from utils import process_tweet, lookup
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

In [2]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [3]:
# pegando todos os tweets positivos e negativos
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# divide os dados em teste e treino
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# cria o rótulo dos dados, 1 para positivo e 0 para negativo
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

# Parte 1: Processar os Dados

Para qualquer projeto de aprendizado de máquina, uma vez que você tenha reunido os dados, o primeiro passo é processá-los para gerar entradas úteis para o seu modelo.

* **Remover ruído**: Primeiro, você vai querer remover o "ruído" dos seus dados — ou seja, eliminar palavras que não dizem muito sobre o conteúdo. Isso inclui todas as palavras comuns como “eu, você, é, está, etc.”, que não nos dão informação suficiente sobre o sentimento.
* Também vamos remover códigos de ações da bolsa de valores, símbolos de retweet, hyperlinks e hashtags, porque eles não fornecem muita informação sobre o sentimento.
* Você também deve remover toda a pontuação de um tweet. A razão para isso é que queremos tratar palavras com ou sem pontuação como a mesma palavra, ao invés de considerar "feliz", "feliz?", "feliz!", "feliz," e "feliz." como palavras diferentes.
* Por fim, você deve usar o *stemming* para manter apenas uma variação de cada palavra. Em outras palavras, vamos tratar "motivação", "motivado" e "motivar" de forma semelhante, agrupando todas na mesma raiz “motiv-”.

Usaremos a função `process_tweet()` que faz esse processamento


```python
def process_tweet(tweet):
    '''
    Entrada:
        tweet: uma string contendo um tweet
    Saída:
        tweets_clean: uma lista de palavras contendo o tweet processado
    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # remove tickers da bolsa como $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove textos de retweet do estilo antigo "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # apenas remove o símbolo # da palavra
    tweet = re.sub(r'#', '', tweet)

    # tokeniza os tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove palavras irrelevantes (stopwords)
            word not in string.punctuation):   # remove pontuação
            stem_word = stemmer.stem(word)     # aplica stemming à palavra
            tweets_clean.append(stem_word)

    return tweets_clean



In [4]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# imprimindo tweet processado
print(process_tweet(custom_tweet))

['hello', 'great', 'day', ':)', 'good', 'morn']


## Parte 1.1 Implementando suas funções auxiliares

Para ajudar no treinamento do seu modelo de Naive Bayes, você precisará construir um **dicionário** onde as chaves são tuplas (palavra, rótulo) e os valores são as **frequências correspondentes**.
Note que os rótulos que usaremos aqui são **1 para positivo** e **0 para negativo**.

Você também implementará uma função auxiliar chamada `lookup()` que recebe como entrada o dicionário `freqs`, uma palavra e um rótulo (1 ou 0), e retorna o número de vezes que essa tupla (palavra, rótulo) aparece na coleção de tweets.

Por exemplo: dado uma lista de tweets `["i am rather excited", "you are rather happy"]` e o rótulo 1, a função retornará um dicionário contendo os seguintes pares chave-valor:

```python
{
    ("rather", 1): 2,
    ("happi", 1): 1,
    ("excit", 1): 1
}
```

* Observe que para cada palavra na string fornecida, o mesmo rótulo `1` é atribuído a cada palavra.
* Observe que as palavras "i" e "am" não são salvas, pois foram removidas pela função `process_tweet()` por serem *stopwords*.
* Observe que a palavra "rather" aparece duas vezes na lista de tweets, então seu valor de contagem é `2`.

---

###  Instruções

Crie uma função `count_tweets()` que receba uma lista de tweets como entrada, limpe todos eles e retorne um dicionário.

* A **chave** no dicionário deve ser uma tupla contendo a palavra com *stemming* aplicada e seu rótulo, por exemplo: `("happi", 1)`.
* O **valor** é o número de vezes que essa palavra aparece na coleção de tweets fornecida (um número inteiro).


In [5]:
def count_tweets(result, tweets, ys):
    '''
    Entrada:
        result: um dicionário que será usado para mapear cada par (palavra, rótulo) para sua frequência
        tweets: uma lista de tweets
        ys: uma lista correspondente ao sentimento de cada tweet (0 ou 1)
    Saída:
        result: um dicionário mapeando cada par para sua frequência
    '''

    ### INÍCIO DO CÓDIGO ###
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # define a chave, que é a tupla (palavra, rótulo)
            pair = (word, y)

            # se a chave já existe no dicionário, incrementa o contador
            if pair in result:
                result[pair] += 1

            # senão, se a chave for nova, adiciona ao dicionário com valor 1
            else:
                result[pair] = 1
    ### FIM DO CÓDIGO ###

    return result


#### Codigo acima já implementado no ultimo projeto de análise de sentimento usando regressão logística

In [6]:
#testando função

result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

## Parte 2: Treine seu modelo usando Naive Bayes

**Naive Bayes** é um algoritmo que pode ser utilizado para análise de sentimentos. Ele leva pouco tempo para ser treinado e também realiza previsões rapidamente.

---

#### Como treinar um classificador Naive Bayes?

* A primeira parte do treinamento de um classificador Naive Bayes é identificar o número de **classes** que você possui.
* Você criará uma **probabilidade para cada classe**.
  $P(D_{pos})$ é a probabilidade de que o documento (tweet) seja **positivo**.
  $P(D_{neg})$ é a probabilidade de que o documento (tweet) seja **negativo**.

Use as seguintes fórmulas e armazene os valores em um dicionário:

$$
P(D_{pos}) = \frac{D_{pos}}{D} \tag{1}
$$

$$
P(D_{neg}) = \frac{D_{neg}}{D} \tag{2}
$$

Onde:

* $D$ é o número total de documentos (neste caso, **tweets**),
* $D_{pos}$ é o número total de **tweets positivos**,
* $D_{neg}$ é o número total de **tweets negativos**.


#### **Prior e Logprior**

A **probabilidade a priori** representa a probabilidade subjacente, na população-alvo, de que um tweet seja positivo em comparação com negativo.
Em outras palavras, se não tivéssemos nenhuma informação específica e escolhêssemos um tweet aleatoriamente do conjunto da população, qual seria a chance de ele ser positivo ou negativo?
Essa é a **probabilidade a priori** (ou simplesmente **prior**).

O prior é a razão entre as probabilidades:

$$
\frac{P(D_{pos})}{P(D_{neg})}
$$

Podemos tirar o logaritmo dessa razão para reescalá-la, e chamamos isso de **logprior**:

$$
\text{logprior} = \log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = \log \left( \frac{D_{pos}}{D_{neg}} \right)
$$

Observe que:

$$
\log\left(\frac{A}{B}\right) = \log(A) - \log(B)
$$

Então o logprior também pode ser calculado como a diferença entre dois logaritmos:

$$
\text{logprior} = \log (P(D_{pos})) - \log (P(D_{neg})) = \log (D_{pos}) - \log (D_{neg}) \tag{3}
$$


#### **Probabilidade Positiva e Negativa de uma Palavra**

Para calcular a **probabilidade positiva** e a **probabilidade negativa** de uma palavra específica no vocabulário, usaremos as seguintes entradas:

* \$freq\_{pos}\$ e \$freq\_{neg}\$ são as **frequências** dessa palavra específica na classe positiva ou negativa.
  Em outras palavras, a frequência positiva de uma palavra é o número de vezes que a palavra aparece com o rótulo 1.

* \$N\_{pos}\$ e \$N\_{neg}\$ são o **número total de palavras positivas e negativas** em todos os documentos (ou seja, em todos os tweets), respectivamente.

* \$V\$ é o **número de palavras únicas** em todo o conjunto de documentos, considerando todas as classes, tanto positivas quanto negativas.

Usaremos esses valores para calcular a probabilidade positiva e negativa de uma palavra específica utilizando as fórmulas abaixo:

$$
P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V} \tag{4}
$$

$$
P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V} \tag{5}
$$

Observe que adicionamos o "+1" no numerador para aplicar o **suavização laplaciana**


##### Criar o dicionário `freqs`

* Usando a sua função `count_tweets()`, você pode calcular um dicionário chamado `freqs` que contém todas as frequências.
* Nesse dicionário `freqs`, a chave é a tupla (palavra, rótulo)
* O valor é o número de vezes que essa tupla apareceu.

Usaremos esse dicionário em várias partes desta atividade.


In [7]:
freqs = count_tweets({}, train_x, train_y)

#### Instruções

Dado um dicionário `freqs`, `train_x` (uma lista de tweets) e `train_y` (uma lista de rótulos para cada tweet), implemente um classificador Naive Bayes.

##### Calcular \$V\$

* Você pode calcular o número de palavras únicas que aparecem no dicionário `freqs` para obter seu \$V\$ (você pode usar a função `set`).

##### Calcular \$freq\_{pos}\$ e \$freq\_{neg}\$

* Usando seu dicionário `freqs`, você pode calcular a frequência positiva e negativa de cada palavra, \$freq\_{pos}\$ e \$freq\_{neg}\$.

##### Calcular \$N\_{pos}\$, \$N\_{neg}\$, \$V\_{pos}\$ e \$V\_{neg}\$

* Usando o dicionário `freqs`, você também pode calcular o número total de palavras positivas e o número total de palavras negativas, \$N\_{pos}\$ e \$N\_{neg}\$.
* Da mesma forma, use o dicionário `freqs` para calcular o número total de palavras **únicas** positivas, \$V\_{pos}\$, e o número total de palavras **únicas** negativas, \$V\_{neg}\$.

##### Calcular \$D\$, \$D\_{pos}\$, \$D\_{neg}\$

* Usando a lista `train_y` de rótulos, calcule o número total de documentos (tweets), \$D\$, bem como o número de documentos positivos (tweets), \$D\_{pos}\$, e o número de documentos negativos (tweets), \$D\_{neg}\$.
* Calcule a probabilidade de um documento (tweet) ser positivo \$P(D\_{pos})\$ e a probabilidade de ser negativo \$P(D\_{neg})\$.

##### Calcular o logprior

* O `logprior` é \$log(D\_{pos}) - log(D\_{neg})\$

##### Calcular a verossimilhança logarítmica (log likelihood)

* Por fim, você pode iterar sobre cada palavra do vocabulário, usar sua função `lookup` para obter as frequências positivas \$freq\_{pos}\$ e negativas \$freq\_{neg}\$ para aquela palavra específica.
* Calcule a probabilidade positiva de cada palavra \$P(W\_{pos})\$ e a probabilidade negativa \$P(W\_{neg})\$ usando as equações (4) e (5):

$P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V} \tag{4}$
$P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V} \tag{5}$

**Nota:** Vamos usar um dicionário para armazenar as verossimilhanças logarítmicas de cada palavra. A chave será a palavra, e o valor será a log likelihood dessa palavra.

* Você pode então calcular a `loglikelihood`:
  \$log \left( \frac{P(W\_{pos})}{P(W\_{neg})} \right)\$.


In [8]:
def treinar_naive_bayes(freqs, train_x, train_y):
    '''
    Entrada:
        freqs: dicionário de (palavra, rótulo) para a frequência com que a palavra aparece
        train_x: uma lista de tweets
        train_y: uma lista de rótulos correspondentes aos tweets (0,1)
    Saída:
        logprior: o log do prior. (equação 3 acima)
        loglikelihood: a verossimilhança logarítmica da equação de Naive Bayes. (equação 6 acima)
    '''
    loglikelihood = {}
    logprior = 0

    # calcular V, o número de palavras únicas no vocabulário
    vocab = set([par[0] for par in freqs.keys()])
    V = len(vocab)

    # calcular N_pos, N_neg, V_pos, V_neg
    N_pos = N_neg = V_pos = V_neg = 0
    for par in freqs.keys():
        # se o rótulo for positivo (maior que zero)
        if par[1] > 0:
            # incrementa o número de palavras positivas únicas
            V_pos += 1

            # incrementa o número total de palavras positivas com a frequência do par (palavra, rótulo)
            N_pos += freqs[par]

        # senão, o rótulo é negativo
        else:
            # incrementa o número de palavras negativas únicas
            V_neg += 1

            # incrementa o número total de palavras negativas com a frequência do par (palavra, rótulo)
            N_neg += freqs[par]

    # calcular D, o número total de documentos
    D = len(train_y)

    # calcular D_pos, o número de documentos positivos
    D_pos = len(list(filter(lambda x: x > 0, train_y)))

    # calcular D_neg, o número de documentos negativos
    D_neg = len(list(filter(lambda x: x <= 0, train_y)))

    # calcular o logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # para cada palavra no vocabulário...
    for palavra in vocab:
        # obtém a frequência positiva e negativa da palavra
        freq_pos = lookup(freqs, palavra, 1)
        freq_neg = lookup(freqs, palavra, 0)

        # calcula a probabilidade de a palavra ser positiva e negativa
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calcula a verossimilhança logarítmica da palavra
        loglikelihood[palavra] = np.log(p_w_pos / p_w_neg)

    ### FIM DO CÓDIGO ###

    return logprior, loglikelihood


In [9]:
logprior, loglikelihood = treinar_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
9067


### Parte 3: Teste seu Naive Bayes

Agora que temos o `logprior` e o `loglikelihood`, podemos testar a função de Naive Bayes fazendo previsões em alguns tweets!

#### Implemente `naive_bayes_predicao`

**Instruções**:
Implemente a função `naive_bayes_predicao` para fazer previsões em tweets.

* A função recebe como entrada o `tweet`, o `logprior` e o `loglikelihood`.
* Ela retorna a **probabilidade** de que o tweet pertença à classe positiva ou negativa.
* Para cada tweet, some as verossimilhanças logarítmicas (`loglikelihood`) de cada palavra contida no tweet.
* Também adicione o `logprior` a essa soma para obter o sentimento previsto daquele tweet.

$p = \text{logprior} + \sum_i^N (\text{loglikelihood}_i)$

---

#### Observação

Note que calculamos o prior (logprior) a partir dos dados de treino, e que os dados de treino estão igualmente divididos entre rótulos positivos e negativos (4000 tweets positivos e 4000 tweets negativos). Isso significa que a razão entre positivos e negativos é 1, e portanto o `logprior` é 0.

O valor `0.0` significa que, ao adicionarmos o `logprior` à `loglikelihood`, estamos apenas somando zero. No entanto, lembre-se de **sempre incluir o logprior**, pois, quando os dados **não estiverem perfeitamente balanceados**, o logprior terá um valor diferente de zero.


In [10]:
def naive_bayes_predicao(tweet, logprior, loglikelihood):
    '''
    Entrada:
        tweet: uma string
        logprior: um número
        loglikelihood: um dicionário que mapeia palavras para números
    Saída:
        p: a soma de todas as verossimilhanças logarítmicas de cada palavra do tweet (se encontrada no dicionário) + o logprior (um número)
    '''
    ### INÍCIO DO CÓDIGO (SUBSTITUA INSTÂNCIAS DE 'None' PELO SEU CÓDIGO) ###

    # processa o tweet para obter uma lista de palavras
    word_l = process_tweet(tweet)

    # inicializa a probabilidade com zero
    p = 0

    # adiciona o logprior
    p += logprior

    # percorre cada palavra na lista de palavras
    for word in word_l:
        # verifica se a palavra existe no dicionário de loglikelihood
        if word in loglikelihood:
            # adiciona a verossimilhança logarítmica da palavra à probabilidade
            p += loglikelihood[word]

    ### FIM DO CÓDIGO ###

    return p

In [14]:
# Experimente com seu próprio tweet
my_tweet = 'She smiled .'
p = naive_bayes_predicao(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1.5705535492750415


#### Implemente `test_naive_bayes`

**Instruções**:

* Implemente a função `test_naive_bayes` para verificar a **acurácia** de suas previsões.
* A função recebe como entrada `test_x`, `test_y`, `log_prior` e `loglikelihood`.
* Ela retorna a **acurácia** do seu modelo.
* Primeiro, use a função `naive_bayes_predict` para fazer previsões para cada tweet em `test_x`.


In [15]:
# UNQ_C6 (IDENTIFICADOR ÚNICO DA CÉLULA, NÃO EDITAR)
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Entrada:
        test_x: uma lista de tweets
        test_y: os rótulos correspondentes para a lista de tweets
        logprior: o logprior
        loglikelihood: um dicionário com as verossimilhanças (loglikelihoods) de cada palavra
    Saída:
        accuracy (acurácia): (número de tweets classificados corretamente) / (número total de tweets)
    """
    accuracy = 0  # retorne isso corretamente

    ### INÍCIO DO CÓDIGO (SUBSTITUA AS INSTÂNCIAS DE 'None' PELO SEU CÓDIGO) ###
    y_hats = []
    for tweet in test_x:
        # se a predição for > 0
        if naive_bayes_predicao(tweet, logprior, loglikelihood) > 0:
            # a classe prevista é 1
            y_hat_i = 1
        else:
            # caso contrário, a classe prevista é 0
            y_hat_i = 0

        # adicione a classe prevista à lista y_hats
        y_hats.append(y_hat_i)

    # o erro é a média dos valores absolutos das diferenças entre y_hats e test_y
    error = np.mean(np.absolute(y_hats - test_y))

    # A acurácia é 1 menos o erro
    accuracy = 1 - error

    ### FIM DO CÓDIGO ###

    return accuracy


In [16]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9940


In [17]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    # print( '%s -> %f' % (tweet, naive_bayes_predict(tweet, logprior, loglikelihood)))
    p = naive_bayes_predicao(tweet, logprior, loglikelihood)
#     print(f'{tweet} -> {p:.2f} ({p_category})')
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.15
I am bad -> -1.30
this movie should have been great. -> 2.14
great -> 2.13
great great -> 4.27
great great great -> 6.40
great great great great -> 8.54


In [19]:
my_tweet = 'you are bad :('
naive_bayes_predicao(my_tweet, logprior, loglikelihood)

np.float64(-8.808571266641943)

### Parte 4: Filtrar palavras por razão entre contagens positivas e negativas

* Algumas palavras aparecem mais frequentemente em tweets positivos do que outras, podendo ser consideradas "mais positivas". Da mesma forma, outras podem ser consideradas mais negativas.

* Uma maneira de definir o nível de positividade ou negatividade, sem calcular a verossimilhança (log likelihood), é comparar a frequência positiva e negativa da palavra.

  * Observe que também podemos usar os cálculos de log likelihood para comparar a positividade ou negatividade relativa das palavras.

* Podemos calcular a razão entre as frequências positivas e negativas de uma palavra:

  $\text{razão} = \frac{\text{contagem positiva} + 1}{\text{contagem negativa} + 1}$

* Uma vez que podemos calcular essas razões, também conseguimos filtrar um subconjunto de palavras que tenham uma razão mínima de positividade/negatividade ou maior.

* De forma semelhante, podemos filtrar palavras com uma razão máxima de positividade/negatividade ou menor (palavras que são pelo menos tão negativas, ou até mais negativas, do que um determinado limite).

---

### Implemente `get_ratio()`

* Dado o dicionário `freqs` com as palavras, e uma palavra específica:

  * Use `lookup(freqs, word, 1)` para obter a contagem positiva da palavra.
  * Use também `lookup(freqs, word, 0)` para obter a contagem negativa da palavra.
* Calcule a razão entre a contagem positiva e negativa com a fórmula:

$\text{razão} = \frac{\text{palavras positivas} + 1}{\text{palavras negativas} + 1}$

Onde `palavras positivas` e `palavras negativas` são as frequências da palavra nas respectivas classes.

---

#### Exemplo de tabela:

| **Palavras** | Contagem Positiva | Contagem Negativa |
| ------------ | ----------------- | ----------------- |
| glad         | 41                | 2                 |
| arriv        | 57                | 4                 |
| :(           | 1                 | 3663              |
| :-(          | 0                 | 378               |


In [20]:
def get_ratio(freqs, word):
    '''
    Entrada:
        freqs: dicionário contendo as palavras

    Saída: um dicionário com as chaves 'positive' (positiva), 'negative' (negativa) e 'ratio' (razão).
        Exemplo: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}

    ### INÍCIO DO CÓDIGO ###
    # usa a função lookup() para encontrar a contagem positiva da palavra (indicada pelo inteiro 1)
    pos_neg_ratio['positive'] = lookup(freqs, word, 1)

    # usa a função lookup() para encontrar a contagem negativa da palavra (indicada pelo inteiro 0)
    pos_neg_ratio['negative'] = lookup(freqs, word, 0)

    # calcula a razão entre as contagens positivas e negativas da palavra
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1) / (pos_neg_ratio['negative'] + 1)
    ### FIM DO CÓDIGO ###

    return pos_neg_ratio


In [21]:
get_ratio(freqs, 'happi')

{'positive': 161, 'negative': 18, 'ratio': 8.526315789473685}

Aqui está a tradução da instrução para implementar a função `get_words_by_threshold`:

---

### Implemente `get_words_by_threshold(freqs, label, threshold)`

* Se definirmos o `label` como `1`, então iremos buscar por todas as palavras cuja **razão entre contagens positiva/negativa** seja **igual ou maior** que o limiar (`threshold`).
* Se definirmos o `label` como `0`, então iremos buscar por todas as palavras cuja **razão entre contagens positiva/negativa** seja **igual ou menor** que o limiar (`threshold`).
* Use a função `get_ratio()` para obter um dicionário contendo a contagem positiva, a contagem negativa e a razão positiva/negativa de uma palavra.
* Adicione um dicionário a uma lista, onde a **chave** é a **palavra** e o **valor** é o dicionário `pos_neg_ratio` retornado pela função `get_ratio()`.

Um exemplo de par chave-valor seria:

```python
{'happi':  # palavra
    {'positive': 10, 'negative': 20, 'ratio': 0.5}  # estatísticas da palavra
}
```



In [23]:
def get_words_by_threshold(freqs, label, threshold):
    '''
    Entrada:
        freqs: dicionário de palavras
        label: 1 para positivo, 0 para negativo
        threshold (limiar): razão que será usada como critério para incluir uma palavra no dicionário retornado

    Saída:
        word_set: dicionário contendo a palavra e informações sobre sua contagem positiva, contagem negativa e razão positiva/negativa.
        exemplo de par chave-valor:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}

    ### INÍCIO DO CÓDIGO ###
    for key in freqs.keys():
        word, _ = key

        # obtém a razão positivo/negativo da palavra
        pos_neg_ratio = get_ratio(freqs, word)

        # se o rótulo for 1 (positivo) e a razão for maior ou igual ao limiar...
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:

            # adiciona a razão ao dicionário
            word_list[word] = pos_neg_ratio

        # se o rótulo for 0 (negativo) e a razão for menor ou igual ao limiar...
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
            # adiciona a razão ao dicionário
            word_list[word] = pos_neg_ratio

        # caso contrário, não inclui a palavra (não faz nada)

    ### FIM DO CÓDIGO ###
    return word_list


In [24]:
# Testando função
get_words_by_threshold(freqs, label=0, threshold=0.05)

{':(': {'positive': 1, 'negative': 3663, 'ratio': 0.0005458515283842794},
 ':-(': {'positive': 0, 'negative': 378, 'ratio': 0.002638522427440633},
 'zayniscomingbackonjuli': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '26': {'positive': 0, 'negative': 20, 'ratio': 0.047619047619047616},
 '>:(': {'positive': 0, 'negative': 43, 'ratio': 0.022727272727272728},
 'lost': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '♛': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 '》': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 'beli̇ev': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'wi̇ll': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'justi̇n': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｓｅｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｍｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776}}

In [27]:
get_words_by_threshold(freqs, label=1, threshold=10)

{'followfriday': {'positive': 23, 'negative': 0, 'ratio': 24.0},
 'commun': {'positive': 27, 'negative': 1, 'ratio': 14.0},
 ':)': {'positive': 2847, 'negative': 2, 'ratio': 949.3333333333334},
 'flipkartfashionfriday': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':d': {'positive': 498, 'negative': 0, 'ratio': 499.0},
 ':p': {'positive': 104, 'negative': 0, 'ratio': 105.0},
 'influenc': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':-)': {'positive': 543, 'negative': 0, 'ratio': 544.0},
 "here'": {'positive': 20, 'negative': 0, 'ratio': 21.0},
 'youth': {'positive': 14, 'negative': 0, 'ratio': 15.0},
 'bam': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'warsaw': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'shout': {'positive': 11, 'negative': 0, 'ratio': 12.0},
 ';)': {'positive': 22, 'negative': 0, 'ratio': 23.0},
 'stat': {'positive': 51, 'negative': 0, 'ratio': 52.0},
 'arriv': {'positive': 57, 'negative': 4, 'ratio': 11.6},
 'via': {'positive': 60, 'negative': 1, 

In [28]:
my_tweet = 'I am happy because I am learning :)'

p = naive_bayes_predicao(my_tweet, logprior, loglikelihood)
print(p)

9.564346021948712
